In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd
from matplotlib import pyplot as plt
import datetime
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler


# Lectura de datos y transformacion de la fecha
mydata = pd.read_csv("shib_data.csv")
mydata['date'] = pd.to_datetime(mydata.snapped_at, format='%Y-%m-%d')

# Reordeenacion y refactorización de los datos
# Separo las fechas para normalizar
mydataDate = mydata[['date']]
mydata = mydata[['price', 'market_cap', 'total_volume']]

# Normalizacion
scaler = MinMaxScaler()
scaler = scaler.fit(mydata)
mydata = scaler.transform(mydata)

# Union de las fechas para separar los conjuntos de entrenamiento y test por fecha
mydata = pd.DataFrame(data=mydata, columns=['price', 'market_cap', 'total_volume'])
mydata = pd.concat([mydataDate, mydata], axis=1)

# Los datos de salida seran los resultados del dia siguiente
mydata['next_day_price'] = mydata[['price']]
for i in mydata.index-1:
  mydata['next_day_price'][i] = mydata['price'][i+1]

# Se crean los conjuntos de entrenamiento y test con sus entradas y salidas
train, test = mydata.loc[mydata.date <= '2021-10-31'], mydata.loc[mydata.date > '2021-10-31']
trainInputs, trainOutputs = train[['price', 'market_cap', 'total_volume']], train[['next_day_price']]
testInputs, testOutputs = test[['price', 'market_cap', 'total_volume']], test[['next_day_price']]

# Implementacion del modelo
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(units=128, input_shape=(443,3)))
model.add(tf.keras.layers.Dense(units=1))
model.compile(loss="mean_squared_error", optimizer = tf.keras.optimizers.Adam(learning_rate=0.001))

callback = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=100)
history = model.fit(trainInputs, trainOutputs, epochs=100, batch_size= 30)



/var/folders/7f/bg4k796x2118xz85gcn87z8h0000gn/T/ipykernel_1170/3822009084.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mydata['next_day_price'][i] = mydata['price'][i+1]


Epoch 1/100


ValueError: in user code:

    File "/usr/local/lib/python3.9/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.9/site-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.9/site-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.9/site-packages/keras/engine/training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.9/site-packages/keras/engine/input_spec.py", line 263, in assert_input_compatibility
        raise ValueError(f'Input {input_index} of layer "{layer_name}" is '

  